# Наследование и полиморфизм

Часто один и тот же алгоритм может быть применен к различным структурам. Мы уже видели, как для того, чтобы более эффективно использовать написанный код, использовались шаблонные функции и перегруженные функции. Подобный подход часто подходит для обработки изоморфных данных (например нахождение масксимума в любой коллекции). 

Однако это не единственный способ обобщения алгоритмов работы с данными. С++ также поддерживает концепцию наследования и полиморфизма.

Наследование - это прием, когда для класса А создается "родственный" ему класс В, который обладает такой же функциональностью, что и класс А + какая-то дополнительая функциональность.

In [1]:
#include <iostream>
#include <cmath>
#include <vector>
#include <map>

In [2]:
class Parent {
public:
    void doX() {
        std::cout << "I do X" << std::endl;
    }
};

In [3]:
class Child : public Parent /*указываем, что Child - наследник класса Parent*/  {
public:
    void doY() {
        std::cout << "I do Y" << std::endl;
    }
};

In [4]:
Parent p;
p.doX(); 

I do X


(void) @0x7fff23b69a90


In [5]:
Child c;
c.doX(); // класс Child умеет все то, что умеет класс Parent
c.doY(); // А также имеет дополнительную функциональность, которую добавит программист

I do X
I do Y


(void) @0x7fff23b69a90


Полиморфизм - способность одинаково работать с разными типами данными, имеющих общего предка.

In [6]:
Parent * n = new Parent();
n->doX(); // Ничего не обычного

I do X


(void) @0x7fff23b69a90


In [7]:
Parent * q = new Child(); // Несмотря на то, что q указывает на тип Parent, мы можем записать по этому указателю
// объект с настоящим типом Child. На тип Child можно смотреть как на тип Parent с дополнительной функциональностью

(Parent *) 0x27af780


In [8]:
q->doX(); // можно вызвать все методы, которые объявлены внутри Parent

I do X


(void) @0x7fff23b69a90


In [9]:
q->doY(); // Насмотря на то, что на самом деле q указывает на объект типа Child, нельзя вызвать метод doY
// так как указатель объявлен как Parent * , то от объекта Child мы можем вызывать только те методы, что достались
// ему от Parent

input_line_18:2:5: error: no member named 'doY' in 'Parent'; did you mean 'doX'?
 q->doY(); // Насмотря на то, что на самом деле q указывает на объект типа Child, нельзя вызвать метод doY
    ^~~
    doX
input_line_4:3:10: note: 'doX' declared here
    void doX() {
         ^


ename: evalue

Возникает логичный вопрос - зачем же тогда требуется полиморфизм и наследование? Ответ заключается в том, что кроме обычных функций, которые вызываются в зависимости от того, какого типа указатель, существуют, так называемые, виртуальные функции, которые вызываются в зависимости от того, на какой реально объект указывает указатель.

In [2]:
class One {
public:
    // виртуальная функция - вызывается в зависимости от объекта
    virtual /*ключевое слово для виртуальной функции*/ void printX() { 
        std::cout << "X from One" << std::endl;
    }
    
    // обычная функция - вызывается в зависимости от типа указателя
    void printY() {
        std::cout << "Y from One" << std::endl;
    }
};

In [3]:
class Two : public One {
public:
    virtual void printX() { // перегружаем виртуальную функцию doX класса N
        std::cout << "X from Two" << std::endl;
    }
    
    // перегружаем обычную функцию doY класса N
    void printY() {
        std::cout << "Y from Two" << std::endl;
    }
};

In [4]:
One * n = new One();
Two * m = new Two();

(Two *) 0x282ca00


In [5]:
n->printX(); // так как в n объект типа N, то вызовется X from N
n->printY(); // так как n объявлен как N * , то вызовется Y from N
// пока ничего необычного

X from One
Y from One


(void) @0x7ffda1ed3ba0


In [6]:
m->printX();
m->printY();
// сейчас все также предсказуему

X from Two
Y from Two


(void) @0x7ffda1ed3ba0


In [7]:
One * k = m; // положим в указатель типа N* объект типа M

(One *) 0x282ca00


In [8]:
k->printX(); // так как в k теперь лежит объект типа M и doX - виртуальная фукнция, то вызовется doX класса M
k->printY(); // несмотря на то, что в k лежит объект типа M, вызовется та функция doY, которая досталась
// M в наследство от N

X from Two
Y from One


(void) @0x7ffda1ed3ba0


In [9]:
// Однако сама функциональность никуда не делать - тип объекта можно вернуть обратно к исходному с помощью
// dynamic_cast
Two * repaired = dynamic_cast<Two*>(k);
repaired->printX();
repaired->printY();

X from Two
Y from Two


(void) @0x7ffda1ed3ba0


In [10]:
// dynamic_cast корректно сработает, только если типы совместимы 
// настоящий объект типа One привести к Two не получится
Two * foo = dynamic_cast<Two*>(n);
std::cout << foo << std::endl; // в объекте foo будет пустой указатель

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fd2031c66e0


Таким образом, мы может определять поведение какого-то объекта в зависимости от того, какой объект мы на самом деле в него положили.

Важным моментом, что мы можем вызывать у таких объектов только те методы, что определены внутри класса-родителя. (в такой ситуации говорят, что объекты имеют общий интерфейс доступа - интерфейс, предоставляемый классов-родителем).

Для демонстрации пользы от подобной функциональности, рассмотрим, что у нас есть множество геометрических фигур (треугольники, прямоугольники и круги) и мы хотим посчитать общую площадь всех фигур.

In [2]:
class Shape { // родительский класс для любой фигуры
public:
    // виртуальная функция площади для того, чтобы функция подсчета площади выбиралась в зависимости от 
    // конкретной фигуры
    virtual double area() { 
        return 0; // так как данный класс не указывает какой-то конкретный класс
        // то будем считать, что его площадь 0
    }
};

In [3]:
class Rectangle : public Shape /*показываем, что прямоугольник является фигурой*/ {
public:
    Rectangle(double w, double h) {
        width = w;
        height = h;
    } // расширяем функциональность "абстрактной" фигуры, указывая, что у прямоугольника есть высота и ширина
    
    virtual double area() {
        return width * height; // площадь прямоугольника
    }
    
private:
    double width, height;
};

In [4]:
class Triangle : public Shape {
public:
    Triangle(double a, double b, double c) { // стороны треугольника
        this->a = a;
        this->b = b;
        this->c = c;
    }
    
    virtual double area() {
        double p = (a + b + c) / 2;
        double skArea = p*(p-a)*(p-b)*(p-c); // Герон
        if(skArea < 0) {
            return 0;
        } else {
            return sqrt(skArea); // функция из cmath
        }
    }
    
private:
    double a, b, c;
};

In [5]:
class Circle : public Shape {
public:
    Circle(double r) {
        radius = r;
    }
    
    virtual double area() {
        return M_PI * radius * radius; // M_PI константа из cmath для числа Пи
    }
    
private:
    double radius;
};

In [6]:
std::vector<Shape*> figures; // массив из любых фигур

figures.push_back(new Rectangle(3, 4));
figures.push_back(new Triangle(3, 4, 5)); 
figures.push_back(new Circle(6));
figures.push_back(new Rectangle(7, 2));
figures.push_back(new Rectangle(2, 1));
figures.push_back(new Triangle(3, 4, 2));
figures.push_back(new Circle(2));
figures.push_back(new Circle(3));
// можно добавлять любые фигуры

(void) @0x7ffddd4a4240


In [7]:
double Area = 0;

for(int i = 0; i < figures.size(); i++) {
    Area += figures[i]->area(); // полиморфно складываем площади всех фигур
}

In [8]:
std::cout << Area << std::endl;

190.843


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fd992cf56e0


In [9]:
// Пример: нахождение максимальной площади среди фигур каждого типа
std::map<std::string, double> areas;
areas["Rectangle"] = 0;
areas["Circle"] = 0;
areas["Triangle"] = 0;

for(Shape * sh : figures) {
    if(dynamic_cast<Rectangle*>(sh) != nullptr) {
        
        areas["Rectangle"] = std::max(areas["Rectangle"], sh->area());
    
    } else if (dynamic_cast<Triangle*>(sh) != nullptr) {
    
        areas["Triangle"] = std::max(areas["Triangle"], sh->area());
    
    } else if(dynamic_cast<Circle*>(sh) != nullptr) {
        
        areas["Circle"] = std::max(areas["Circle"], sh->area());
    }
}

for(auto f : areas) {
    std::cout << f.first << ": " << f.second << std::endl;
}

Circle: 113.097
Rectangle: 14
Triangle: 6


In [2]:
// В С++ есть три модификатора доступа для доступа к данным
// public - данные доступны отовсюду
// private - данные доступны только в этом классе. То есть даже в потомках не будет доступа к ним
// protected - аналогичен private, однако теперь данные доступны потомкам
class First {
public:
    First() {
        a = 1;
        b = 2;
        c = 3;
    }
    virtual void print() {
        std::cout << a << std::endl; // сам класс имеет доступ ко всем своим полям
        std::cout << b << std::endl;
        std::cout << c << std::endl;
    }
    int a;
protected:
    int b;
private:
    int c;
};

class Second : public First {
public:
    Second() : First() {}
    virtual void print() {
        std::cout << a << std::endl; // нет ошибки
        std::cout << b << std::endl; // нет ошибки
        std::cout << c << std::endl; // ошибка - доступа к private у наследников нет
    }
};

First f;
std::cout << f.a << std::endl; // нет ошибки
std::cout << f.b << std::endl; // ошибка - извне нет доступа к protected
std::cout << f.c << std::endl; // ошибка - извне нет доступа к private

input_line_4:31:22: error: 'c' is a private member of 'First'
        std::cout << c << std::endl; // ошибка - доступа к private у наследников нет
                     ^
input_line_4:21:9: note: declared private here
    int c;
        ^
input_line_4:37:16: error: 'b' is a protected member of 'First'
std::cout << f.b << std::endl; // ошибка - извне нет доступа к protected
               ^
input_line_4:19:9: note: declared protected here
    int b;
        ^
input_line_4:38:16: error: 'c' is a private member of 'First'
std::cout << f.c << std::endl; // ошибка - извне нет доступа к private
               ^
input_line_4:21:9: note: declared private here
    int c;
        ^


ename: evalue

# Интерпретатор

С помощью подобного подхода можно очень удобно разделять логику работы вашей программы. 
Для еще одного примера, напишем небольшой интерпретатор собственного языка программиования.

В нашем языке программирования есть всего 7 операций: 

set - устанавливает значение переменной

add - складывает

subl - вычитает

mult - умножает

sub - делит

pow - возводит в степень

print - выводит на экран

Все команды имеют вид:

[command] [target] [value]

print также поддерживает вывод произвольного количества аргументов: print a b c ...


In [ ]:
#include <iostream>
#include <string>
#include <map>
#include <cmath>
#include <sstream>

class Handler;

class Interpreter {
public:
    Interpreter();
    ~Interpreter();
    void run(std::string code); // Реализуем данные фукнции после объявление обработчиков, так как
    // иначе они не будут о них ничего знать

    std::stringstream codeStream; // тоже самое, что и std::cin \ std::cout, точно чтение происходит из строки а не с клавиатуры
    std::map<std::string, int> variables; // множество всех переменных
    std::map<std::string, Handler*> handlers; // множество всех обработчиков
};


class Handler { // общий предок для обработчиков
public:
    Handler(Interpreter * inter) {
        interpretator = inter;
    }
    virtual void handle() {}

    std::pair<std::string, int> getAgruments() { // считать оба аргумента - общая функция для двух обработчиков
        std::string target, value;
        interpretator->codeStream >> target >> value;
        if(value[0] >= '0' and value[0] <= '9') { // если просто число
            return std::make_pair(target, std::stoi(value));
        } else { // если переменная
            return std::make_pair(target, interpretator->variables[value]);
        }
    };
protected: // секция protected аналогична private, но все ее содержимое будет доступно наследникам
    // здесь используется именно protected, а не private или public, так как 
    // мы не хотим раскрывать внутреннего устройства объекта (нарушать инкапсуляцию)
    // но при этом хотим, чтобы все наследники могли свободно пользоваться данным объектом
    Interpreter * interpretator;
};

class SetHandler : public Handler { // обработчик установки значения переменной
public:
    SetHandler(Interpreter *inter) : Handler(inter) /*конструктор родителя*/ {} // конструктор необходимо прописать явно
    // однако не обязательно его еще реализовывать - достаточно вызвать конструктор родителя.

    virtual void handle() {
        auto args = getAgruments();
        interpretator->variables[args.first] = args.second;
    }
};

class AddHandler : public Handler {
public:
    AddHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle() {
        auto args = getAgruments();
        interpretator->variables[args.first] += args.second;
    }
};

class SubHandler : public Handler {
public:
    SubHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle() {
        auto args = getAgruments();
        interpretator->variables[args.first] -= args.second;
    }
};

class MultHandler : public Handler {
public:
    MultHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle()  {
        auto args = getAgruments();
        interpretator->variables[args.first] *= args.second;
    }
};

class DivHandler : public Handler {
public:
    DivHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle() {
        auto args = getAgruments();
        interpretator->variables[args.first] /= args.second;
    }
};

class PowHandler : public Handler { // возведение в степень
public:
    PowHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle() {
        auto args = getAgruments();
        interpretator->variables[args.first] = (int)pow(interpretator->variables[args.first], args.second);
    }
};

class PrintHandler : public Handler { // печать на экран
public:
    PrintHandler(Interpreter *inter) : Handler(inter) {}

    virtual void handle() {
        std::string variables, current;
        std::getline(interpretator->codeStream, variables);
        std::stringstream ss;
        ss << variables;
        while(ss >> current) {
            if(current[0] >= '0' and current[0] <= '9') std::cout << current << ' '; // Если просто число
            else std::cout << interpretator->variables[current] << ' '; // Если переменная
        }
        std::cout << std::endl;
    }
};

Interpreter::Interpreter() {
    handlers["set"] = new SetHandler(this); // установка соотвествующий обработчкиков для операций
    handlers["add"] = new AddHandler(this);
    handlers["subl"] = new SubHandler(this);
    handlers["mult"] = new MultHandler(this);
    handlers["div"] = new DivHandler(this);
    handlers["pow"] = new PowHandler(this);
    handlers["print"] = new PrintHandler(this);
}

Interpreter::~Interpreter() {
    for(auto h : handlers) {
        delete h.second; // по окончанию, необходимо освбодить память
    }
}

void Interpreter::run(std::string code) {
    codeStream.str(code);
    std::string command;
    while(codeStream >> command) { // считываем все комманды в коде
        handlers[command]->handle(); // для соответствующей команды вызываем соответствующий обработчик
    }
    std::stringstream().swap(codeStream); // очищаем ввод
}

int main() {
    Interpreter Inter;

    std::string hello = "print 42";

    Inter.run(hello);

    std::string adding = "\
set a 10 \n \
set b 20 \n \
add a b \n \
print a b \n \
";
    Inter.run(adding);

    std::string code = "\
set a 10 \n\
set b 20 \n \
set c 1 \n \
mult c a \n \
mult c b \n \
\
set k c \n \
div k 10 \n \
pow k 5 \n \
\
set abcd k \n \
div abcd 2 \n \
div abcd a \n \
\
set f 0 \n \
subl f a \n \
subl f b \n \
\
set summ 0 \n \
add summ a \n \
add summ b \n \
add summ c \n \
add summ k \n \
add summ f \n \
\
print a b c k f summ abcd \n \
";
    Inter.run(code);
    return 0;
}

Как можно заметить, весь код интерпретатора заключается в том, что на каждой строке мы вызывает один из обработчиков, который определяется автоматически исходя из команды.

Каждый обработчик будет получать аргументы операции. Таким образом мы можем вынести этот метод в родительский класс для обработчиков. В итоге, необходимо написать необходимое количество классов на каждый оператор и реализовать в нем всего одну функцию. 

Таким образом, код становится понятнее и его гораздо удобнее расширять в дальнейшем: для добавления новой команды, не нужно трогать код интерпретатора - достаточно написать новый класс с одной функцией, который будет обрабатывать эту новую операцию и подключить ее.

Аналогичным образом можно, к примеру, обрабатывать аргументы командной строки.

Представьте, насколько сложнее и страшнее оказался бы код, если бы он был реализован с помощью switch или if-else. (Особенно, если бы Вам пришлось реализовывать вложенные конструкции, такие как цикл, например)

# Декорирование

Еще одним примером эффективного использования полиморфизма - декорирование объектов.

Предположим, что у Вас есть какой-то объект, который выполняет некоторую полезную работу. И вы хотите добавить к этому действию, какое то дополнительное - к примеру, вести лог событий или, к примеру, сингализировать системе о начале и завершения работы.

In [ ]:
#include <iostream>

class IWorker { // буква I - от слова Interface. То есть данный класс сам по себе ничего не делает а только предоставляет
    // интерфейс для потомков
public:
    virtual void doWork(){}
    virtual int getId() { return -1; }
};

class Worker : public IWorker {
public:
    Worker(int id) {
        this->id = id;
    }
    virtual void doWork() {
        for(int i = 0; i < 5; i++) {
            std::cout << "Working ..." << std::endl;
            // Эмулируем серьезную работу
            // в реальной задаче, тут могут быть какие-то серьезные вычисление
            // отправка каких-нибудь данных
            // работы с сетью или что-то подобное
        }
    }

    virtual int getId() {
        return id;
    }

private:
    int id;
};

class LoggerDecorator : public IWorker { // логгер
public:
    LoggerDecorator(IWorker * worker) {
        this->worker = worker; // получаем ссылку на настоящего рабочего
    }

    virtual int getId() { // возвращаем id настоящего работника
        return worker->getId();
    }

    virtual void doWork() { // обрамляем работу реального работника логгированием
        std::cout << "[Log]: Worker with id " << worker->getId() << " start working." << std::endl;
        worker->doWork();
        std::cout << "[Log]: Worker with id " << worker->getId() << " finish working." << std::endl;
    }


private:
    IWorker * worker;
};

class BeautyDecorator : public IWorker { // еще один декоратор, который рисует красивые строчки до и после основной работы
public:
    BeautyDecorator(IWorker * worker) {
        this->worker = worker;
    }

    virtual void doWork()  {
        for(int i = 0; i < 60; i++) std::cout << '=';
        std::cout << std::endl;

        worker->doWork();

        for(int i = 0; i < 60; i++) std::cout << '=';
        std::cout << std::endl;
    }

    virtual int getId()  {
        return worker->getId();
    }


private:
    IWorker * worker;
};

int main() {
    IWorker * one = new Worker(1); // Обычный рабочий
    one->doWork();

    std::cout << std::endl;

    IWorker * two = new LoggerDecorator(new Worker(2)); // оборачиваем рабочего в логгер
    // принцип работы с объектом two никак не поменялся
    // но при этом мы аккуратно добавили новую функциональность, не изменяя изначальный класс рабочего

    two->doWork(); // работаем но теперь с логгированием

    std::cout << std::endl;

    IWorker * three = new BeautyDecorator(new Worker(3)); // аналогично мы можем сделать и с Beauty
    three->doWork();

    std::cout << std::endl;

    // так как все классы - наследники IWorker, то со всеми ними мы можем работать одинаковым образом
    // То есть мы можем оборачивать в произвольное клоичество декораторов
    IWorker * four = new BeautyDecorator(new LoggerDecorator(new Worker(4)));

    four->doWork();

    std::cout << std::endl;

    IWorker * five = new LoggerDecorator(one); // можем оборачивать уже созданные классы, не обязательно создавать новые

    five->doWork();

    std::cout << std::endl;

    return 0;
}

In [ ]:
Вывод:

Working ...
Working ...
Working ...
Working ...
Working ...

[Log]: Worker with id 2 start working.
Working ...
Working ...
Working ...
Working ...
Working ...
[Log]: Worker with id 2 finish working.

============================================================
Working ...
Working ...
Working ...
Working ...
Working ...
============================================================

============================================================
[Log]: Worker with id 4 start working.
Working ...
Working ...
Working ...
Working ...
Working ...
[Log]: Worker with id 4 finish working.
============================================================

[Log]: Worker with id 1 start working.
Working ...
Working ...
Working ...
Working ...
Working ...
[Log]: Worker with id 1 finish working.

В данном примере на не важно, что лежит на самом деле по указателю. Нам важно лишь то, что данный объект умеет совершать две операции: doWork и getId.
 
Таким образом, можно не переделывая всю струкруту проекта добавлять дополнительный функционал к уже существующим объектам